<a href="https://colab.research.google.com/github/Mahjabeenqamar1/TF-IDF-CODE/blob/main/For_TF_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.2
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model, models, layers
import matplotlib.pyplot as plt
%matplotlib inline
from google.colab import files
import time
from tensorflow.keras.utils import Sequence
import random
import math

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.2`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load(img_path):
    return np.load(img_path, allow_pickle=True)

def create_ID_dict(basepath):
  dictionary = {}
  mask_list =[]
  image_list =[]

  for file in glob.glob(basepath+'/**/**.npy', recursive=True):
    if 'r.npy' in file:
      mask_list.append(file)
    elif 'cti.npy' in file:
      image_list.append(file)

  
  dictionary.update({"image":image_list})
  dictionary.update({"mask":mask_list})
  return dictionary


'''class ImageSequence(Sequence):
    def __init__(self, x_set, y_set, batch_size):
    
        self.dictionary = {}
        mask_list =[]
        image_list =[]
        
        self.batch_size = batch_size

        basepath = '/content/drive/My Drive/Sample_Data_for_TF/trv1_p1'

        for file in glob.glob(basepath+'/**/**.npy', recursive=True):
            if 'r.npy' in file:
                mask_list.append(file)
            elif 'cti.npy' in file:
                image_list.append(file)

  
        self.dictionary["image"] = image_list
        self.dictionary["mask"] = mask_list


    def __len__(self):
        return math.ceil(len(self.dictionary["image"]) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.dictionary["image"][idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = self.dictionary["mask"][idx * self.batch_size:(idx + 1) *
        self.batch_size]


        return np.array([load(file_name)[random.randrange(0,50)].reshape(1,512,512) for file_name in batch_x]), 
        np.array([load(file_name)[random.randrange(0,50)].reshape(1,512,512) for file_name in batch_y])'''


def dataset_gen(dictionary, mask):
  image_list= [] 
  mask_list = []
 

  if mask == True:
    n = 0

    for path0, path1 in zip(dictionary['image'],dictionary['mask']):
      image0 = load(path0)
     # print("{} loaded".format(path0))
      image1 = load(path1) #preprocessing has happened befoer hand
     # print("{} loaded".format(path1))
      
      for slice0, slice1 in zip(image0,image1):
        #print("in slice loop")

        slice0= np.reshape(slice0,(1,512,512))
        slice0 = slice0.astype('float32')
        print("slice0 shape: ", slice0.shape)

        slice1= np.reshape(slice1,(1,512,512))
        slice1= slice1.astype('float32')
        print("slice1 shape: ", slice1.shape)
        
        image_list.append(slice0)
        mask_list.append(slice1)
       

      n+=1
      print("{} images have been loaded.".format(str(n)+'/'+str(len(dictionary['image']))))
    
    return image_list, mask_list
  else:
    print("option not possible yet")



In [ ]:
'''Predetermined Variables'''
PATH_tr = '/content/drive/My Drive/Sample_Data_for_TF/trv1_p1'


In [ ]:
''' Loading Data Sets '''
print("loading datasets")

start_time = time.time()
tr_dict = create_ID_dict(PATH_tr)
end_time = time.time()

print("Dictionary Creation Took {} seconds".format(end_time-start_time))

start_time = time.time()
image_list, mask_list = dataset_gen(tr_dict,mask = True)
end_time = time.time()


print("Generators Creation Took {} sec".format(end_time-start_time))


loading datasets
Dictionary Creation Took 0.0014197826385498047 seconds
slice0 shape:  (1, 512, 512)
slice1 shape:  (1, 512, 512)
slice0 shape:  (1, 512, 512)
slice1 shape:  (1, 512, 512)
slice0 shape:  (1, 512, 512)
slice1 shape:  (1, 512, 512)
slice0 shape:  (1, 512, 512)
slice1 shape:  (1, 512, 512)
slice0 shape:  (1, 512, 512)
slice1 shape:  (1, 512, 512)
slice0 shape:  (1, 512, 512)
slice1 shape:  (1, 512, 512)
slice0 shape:  (1, 512, 512)
slice1 shape:  (1, 512, 512)
slice0 shape:  (1, 512, 512)
slice1 shape:  (1, 512, 512)
slice0 shape:  (1, 512, 512)
slice1 shape:  (1, 512, 512)
slice0 shape:  (1, 512, 512)
slice1 shape:  (1, 512, 512)
slice0 shape:  (1, 512, 512)
slice1 shape:  (1, 512, 512)
slice0 shape:  (1, 512, 512)
slice1 shape:  (1, 512, 512)
slice0 shape:  (1, 512, 512)
slice1 shape:  (1, 512, 512)
slice0 shape:  (1, 512, 512)
slice1 shape:  (1, 512, 512)
slice0 shape:  (1, 512, 512)
slice1 shape:  (1, 512, 512)
slice0 shape:  (1, 512, 512)
slice1 shape:  (1, 512, 512)


In [ ]:
'''Model Creation'''
tf.executing_eagerly()
#Define model#
# Define kwargs dictionary#
kwargs = {
  'kernel_size': (3,3),
  'padding': 'same',
  'data_format': 'channels_first'}

#Define lambda functions#
conv = lambda x, filters, strides : layers.Conv2D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.LeakyReLU()(x)

#Define stride-1, stride-2 blocks#
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=2)))

#Define single transpose#
tran = lambda x, filters, strides : layers.Conv2DTranspose(filters=filters, strides=strides, **kwargs)(x)

#Define transpose block#
tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=2)))
concat = lambda a, b : layers.Concatenate(axis=1)([a, b])

#Define Dense Block#
def dense_block(filters,input):
  l1 = conv1(filters, input)
  c1 = concat(input,l1)
  l2 = conv1(filters,c1)
  c2 = concat(c1,l2)
  l3 = conv1(filters, c2)
  c3 = concat(c2,l3)
  l4 = conv1(filters,c3)

  return concat(input,l4)

def td_block(conv1_filters,conv2_filters,input):
  TD = conv1(conv1_filters,conv2(conv2_filters,input))
  print("TD Shape = ", TD.shape)
  DB = dense_block(conv1_filters,TD)
  print("Dense Block Shape: ", DB.shape)
  return DB

def tu_block(conv1_filters,tran2_filters,input,td_input):
  TU = conv1(conv1_filters,tran2(tran2_filters,input))
  print("TU Shape: ", TU.shape)
  C = concat(TU,td_input)
  print("C Shape: ", C.shape)
  DB = dense_block(conv1_filters,C)
  print("Dense Block Shape: ", DB.shape)
  return DB

#Build Model#
# TD = convolutions that train down, DB = Dense blocks, TU = Transpose convolutions that train up, C = concatenation groups.

inputs = Input(shape=(1,512,512))
print("Input Shape: ", inputs.shape)
TD1 = conv2(18,inputs) #basic convolution of stride 2, no maxpool.
print("TD Shape: ", TD1.shape)
DB1 = dense_block(18,TD1)
print("Dense Block Shape: ",DB1.shape )

TD2 = td_block(32,18, DB1)
TD3 = td_block(64,32,TD2)
TD4 = td_block(86,64,TD3)
TD5 = td_block(126,86,TD4)
print("TD5 shape: ", TD5.shape)

TU1 = tu_block(86,126,TD5,TD4)
TU2 = tu_block(64,86,TU1,TD3)
TU3 = tu_block(32,64,TU2,TD2)
TU4 = tu_block(18,32,TU3,TD1)
TU5 = tran2(1, TU4)


logits = layers.Conv2D(filters = 1, **kwargs)(TU5)
print(logits.shape)
model = Model(inputs=inputs, outputs=logits )


optimizer = "adam"
loss = "categorical_crossentropy"
metrics = ['accuracy']
model.compile(optimizer= optimizer,loss = loss, metrics =metrics)


Input Shape:  (None, 1, 512, 512)
TD Shape:  (None, 18, 256, 256)
Dense Block Shape:  (None, 36, 256, 256)
TD Shape =  (None, 32, 128, 128)
Dense Block Shape:  (None, 64, 128, 128)
TD Shape =  (None, 64, 64, 64)
Dense Block Shape:  (None, 128, 64, 64)
TD Shape =  (None, 86, 32, 32)
Dense Block Shape:  (None, 172, 32, 32)
TD Shape =  (None, 126, 16, 16)
Dense Block Shape:  (None, 252, 16, 16)
TD5 shape:  (None, 252, 16, 16)
TU Shape:  (None, 86, 32, 32)
C Shape:  (None, 258, 32, 32)
Dense Block Shape:  (None, 344, 32, 32)
TU Shape:  (None, 64, 64, 64)
C Shape:  (None, 192, 64, 64)
Dense Block Shape:  (None, 256, 64, 64)
TU Shape:  (None, 32, 128, 128)
C Shape:  (None, 96, 128, 128)
Dense Block Shape:  (None, 128, 128, 128)
TU Shape:  (None, 18, 256, 256)
C Shape:  (None, 36, 256, 256)
Dense Block Shape:  (None, 54, 256, 256)
(None, 1, 512, 512)


In [ ]:
image_list = tf.convert_to_tensor(image_list)
mask_list = tf.convert_to_tensor(mask_list)


TypeError: ignored

In [ ]:
#obj= ImageSequence(None, None,1)
model.fit(image_list,mask_list, epochs=1, batch_size=20 )


4/4 [==============================] - 3s 764ms/step - loss: 0.0000e+00 - accuracy: 0.0015
